In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from scipy.stats import norm
from typing import List, Dict
from sklearn.metrics import log_loss, roc_auc_score
from scipy.special import logit

# Data preparation

Прочитаем данные, выделив сразу необходимые нам фичи

In [2]:
full_data = pd.read_csv(
    '../data/data.csv',
    usecols=[
        'date_time',
        'zone_id',
        'banner_id',
        'os_id',
        'country_id',
        'banner_id0',
        'banner_id1',
        'g0',
        'g1',
        'coeff_sum0',
        'coeff_sum1',
        'clicks'
    ],
    parse_dates=['date_time'],
    infer_datetime_format=True
)
full_data.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30,0,0,0,0,1240,0.035016,-7.268846,0,0.049516,-5.369901,1
1,2021-09-26 22:54:49,1,1,0,1,1,0.054298,-2.657477,269,0.031942,-4.449220,1
2,2021-09-26 23:57:20,2,2,0,0,2,0.014096,-3.824875,21,0.014906,-3.939309,1
3,2021-09-27 00:04:30,3,3,1,1,3,0.015232,-3.461357,99,0.050671,-3.418403,1
4,2021-09-27 00:06:21,4,4,1,0,4,0.051265,-4.009026,11464230,0.032005,-2.828797,1


Фичу `date_time` разделим на день и час: одну будем использовать для сплита, а другую для обучения.

In [3]:
full_data['date'] = full_data['date_time'].dt.date
full_data['day_hour'] = full_data['date_time'].dt.hour

Как обычно, выкидваем день `2021-09-01`, а оставшиеся данные делим на трейн и тест.

In [4]:
full_data = full_data[full_data.date != pd.Timestamp('2021-09-01').date()]

full_data_test = full_data[full_data.date == pd.Timestamp('2021-10-02').date()]
full_data_train = full_data[full_data.date != pd.Timestamp('2021-10-02').date()]

Выделим из трейна таргет.

In [5]:
y_train = full_data_train['clicks'].to_numpy()
data_train = full_data_train.drop('clicks', axis=1)
print(f"Train size: {len(data_train)}")

Train size: 13692493


Теперь разберёмся с тестом, на котором будем считать CIPS.

Проверим на `nan` новые фичи, которые в этом задании используем в первый раз.

In [6]:
full_data_test.isna().sum()

date_time        0
zone_id          0
banner_id        0
os_id            0
country_id       0
banner_id0       0
g0              12
coeff_sum0      12
banner_id1       0
g1            4904
coeff_sum1    4904
clicks           0
date             0
day_hour         0
dtype: int64

Как видно, `nan` есть. Удалим строки с `nan`.

Помимо этого, вспоминая об условии задания, удалим строки, в которых `banner_id != banner_id0`.

In [7]:
full_data_test = full_data_test.dropna()

diverged_banners_mask = full_data_test['banner_id'] != full_data_test['banner_id0']
print(f"The number of diverged banners: {diverged_banners_mask.sum()}")

full_data_test = full_data_test[~diverged_banners_mask]

The number of diverged banners: 238404


Осталось отделить таргет фичу, а также создать второй тест, в котором `banner_id` -- это `banner_id1`

In [8]:
y_test = full_data_test['clicks'].to_numpy()
data_test = full_data_test.drop('clicks', axis=1)
print(f"Test size: {len(data_test)}")

data_test_reversed = data_test.copy()
data_test_reversed['banner_id'] = data_test['banner_id1']

Test size: 1885670


Также, следуя решению первой домашки, добавим интеракции

In [9]:
def add_interaction(data: pd.DataFrame, interacting_features: List[str]):
    def create_interaction(row):
        interaction = []
        for feature in interacting_features:
            interaction.append(str(row[feature]))
        return ":".join(interaction)
    new_column_name = ":".join(interacting_features)
    print(f"New column name: {new_column_name}")
    data[new_column_name] = data.apply(create_interaction, axis=1)

In [10]:
interactions = [
    ['banner_id', 'country_id'],
    ['zone_id', 'os_id'],
    ['banner_id', 'os_id'],
    ['banner_id', 'day_hour']
]
for interaction in interactions:
    add_interaction(data_train, interaction)
    add_interaction(data_test, interaction)
    add_interaction(data_test_reversed, interaction)

New column name: banner_id:country_id
New column name: banner_id:country_id
New column name: banner_id:country_id
New column name: zone_id:os_id
New column name: zone_id:os_id
New column name: zone_id:os_id
New column name: banner_id:os_id
New column name: banner_id:os_id
New column name: banner_id:os_id
New column name: banner_id:day_hour
New column name: banner_id:day_hour
New column name: banner_id:day_hour


# Data transformation

Все фичи категориальные, так как в этом задании не используем фичу `campaign_clicks`

In [11]:
categorical_features = [
    'country_id',
    'os_id',
    'zone_id',
    'banner_id',
    'day_hour',
    'banner_id:country_id',
    'zone_id:os_id',
    'banner_id:os_id',
    'banner_id:day_hour'
]

Код по преобразованию данных для модели аналогичен коду из первой домашки:

In [12]:
def create_encoders(data: pd.DataFrame, categorical_features: List[str]):
    encoders = {}
    for feature in categorical_features:
        encoders[feature] = OneHotEncoder(handle_unknown='ignore').fit(data[[feature]])
    return encoders

def transform_data(data: pd.DataFrame, categorical_features: List[str], feature_encoders: Dict):
    transformed_data = []

    for feature in categorical_features:
        encoder = feature_encoders[feature]
        transformed_data.append(encoder.transform(data[[feature]]))

    return sparse.hstack(transformed_data)

# Model training

Для решения логистической регрессии будем использовать не SGD-like `liblinear` solver с `l2` регуляризацией.
Метод эффективно находит решение регрессии для sparse данных. А это как раз наш случай)

In [13]:
def create_model(X, y, C):
    return LogisticRegression(solver='liblinear', penalty='l2', C=C, random_state=42).fit(X, y)

Реализуем подсчёт базовых метрик

In [14]:
def get_score(y_true, y_pred):
    # y_pred.shape == [N, 2].
    # Первый столбец -- вероятности 0 (отсутствия клика)
    # Второй столбец -- вероятность 1 (клика)
    return {
        'log-loss': log_loss(y_true, y_pred),
        'roc-auc': roc_auc_score(y_true, y_pred[:, 1])
    }

Преобразуем данные

In [15]:
feature_encoders = create_encoders(data_train, categorical_features)

X_train = transform_data(data_train, categorical_features, feature_encoders)
X_test = transform_data(data_test, categorical_features, feature_encoders)
X_test_reversed = transform_data(data_test_reversed, categorical_features, feature_encoders)

Обучим лучшую модель из первой домашки (модель с интеракциями и `C=0.1`):

In [16]:
model = create_model(X_train, y_train, C=0.1)

 Посчитаем базовые метрики на последнем дне

In [17]:
y_pred = model.predict_proba(X_test)
best_score = get_score(y_test, y_pred)
print(json.dumps(best_score, sort_keys=True, indent=4))

{
    "log-loss": 0.13218053452125872,
    "roc-auc": 0.8003288993159902
}


# CIPS calculation

Для того чтобы посчитать вероятность показа баннера, нужно оценить следующую вероятность:
```
P(X_0 > X_1) = P(X_0 - X_1 > 0) = 1 - F_{X_0 - X_1}(0)
```
, где `X_0`, `X_1` -- две независимые нормальные величины, которые подбрасываются для выбора банера;
`F_{X_0 - X_1}` -- функция распределения случайной величины `X_0 - X_1`.

Так как `X_0` и `X_1` -- независимые нормальные случайные величины, то их разность -- это тоже нормальная случайная величина с:
* средним `mean(X_0) - mean(X_1)`
* стандартным отклонением: `sqrt(sigma_0**2 + sigma_1**2)`

In [18]:
def get_banner_probability(coeff_sum_target, g_target, coeff_sum_second, g_second):
    return 1. - norm.cdf(0., loc=coeff_sum_target - coeff_sum_second, scale=np.sqrt(g_target**2 + g_second**2) + 1e-9)

In [19]:
old_scores = get_banner_probability(data_test['coeff_sum0'], data_test['g0'], data_test['coeff_sum1'], data_test['g1'])

y_pred0 = model.predict_proba(X_test)
y_pred1 = model.predict_proba(X_test_reversed)
new_coeff_sum0 = logit(y_pred0[:, 1])
new_coeff_sum1 = logit(y_pred1[:, 1])

new_scores = get_banner_probability(new_coeff_sum0, data_test['g0'], new_coeff_sum1, data_test['g1'])

Итоговый CIPS:

In [20]:
cips = np.mean(y_test * np.minimum(new_scores / (old_scores + 1e-9), 10))
print(f"CIPS: {cips}")

CIPS: 0.06490593811412901
